In [15]:
import cv2
import numpy as np
import imutils
import math

#Background for 1st Square Region of Interest(ROI)
bg1=None
#Background for 2nd Square Region of Interest(ROI)
bg2=None
cap=cv2.VideoCapture(0)
noFrame=1
#Co-ordinated for ROI's
roiX1=0
roiY1=100
roiX2=200
roiY2=300
roiP1=500
roiQ1=100
roiP2=700
roiQ2=300

#Location to save output
output='F:\output.avi'
writer=None

#Compute avergae for background till 1st 30 frames
def avgBgr(frame, index, Weight):
    if index==1:
        global bg1
        if bg1 is None:
            bg1=frame.copy().astype('float')
            return
        bg1=cv2.accumulateWeighted(frame, bg1, Weight)
    if index==2:
        global bg2
        if bg2 is None:
            bg2=frame.copy().astype('float')
            return
        bg2=cv2.accumulateWeighted(frame, bg2, Weight)

#Detect hand by getting difference from background and hand that now entered the ROI
def segment(image, index, threshold=30):
    if index==1:
        global bg1
        diff = cv2.absdiff(bg1.astype("uint8"), image)
        thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
        (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(cnts) == 0:
            return
        else:
            segmented = max(cnts, key=cv2.contourArea)
            return (thresholded, segmented)
    if index==2:
        global bg2
        diff = cv2.absdiff(bg2.astype("uint8"), image)
        thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
        (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(cnts) == 0:
            return
        else:
            segmented = max(cnts, key=cv2.contourArea)
            return (thresholded, segmented)
        
def getOutput(hand, index):
    try:
        (thresholded, segmented)=hand
        #Contour Area of hand
        cntArea=cv2.contourArea(segmented)
        if index==1:
            cv2.imshow('thresh1', thresholded)
        if index==2:
            cv2.imshow('thresh2', thresholded)
        #COnvex hull around hand
        hull=cv2.convexHull(segmented)
        areahull = cv2.contourArea(hull)
        arearatio=((areahull-cntArea)/cntArea)*100
        #Moments to get center of Hand
        moments=cv2.moments(segmented)
        if moments['m00']!=0:
            cx=int(moments['m10']/moments['m00'])
            cy=int(moments['m01']/moments['m00'])
        center=(cx,cy)
        cnt = cv2.approxPolyDP(segmented,0.01*cv2.arcLength(segmented,True),True)
        hull = cv2.convexHull(cnt,returnPoints = False)
        #Get defects in hand to identify number generated from hand
        defects = cv2.convexityDefects(cnt,hull)
        mind=0
        maxd=0
        l=0
        #Get area of triangle formed by defects
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(cnt[s][0])
            end = tuple(cnt[e][0])
            far = tuple(cnt[f][0])
            dist = cv2.pointPolygonTest(cnt,center,True)
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            d=(2*ar)/a
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            if angle <= 90 and d>30:
                l += 1
        l+=1
        font = cv2.FONT_HERSHEY_SIMPLEX
        key=cv2.waitKey(1)
        if key==27:
            cv2.destroyAllWindows()
        #Get corresponding digit from fingers, -1 here refers to no output
        if l==1:
            if cntArea<2000:
                return -1
            else:
                if arearatio<12:
                    return 0
                elif arearatio<17.5:
                    return 1

                else:
                    return 1
        elif l==2:
            return 2
        elif l==3:
            return 3
        elif l==4:
            return 4
        elif l==5:
            return 5
        elif l==6:
            return -1
        else :
            return -1
    except:
        pass

while(1):
    check, img=cap.read()
    frame=img.copy()
    frame=cv2.flip(frame, 1)
    frame=imutils.resize(frame, width=700)
    roi1=frame[roiY1:roiY2,roiX1:roiX2]
    roi2=frame[roiQ1:roiQ2,roiP1:roiP2]
    gray1=cv2.cvtColor(roi1, cv2.COLOR_BGR2GRAY)
    blur1=cv2.GaussianBlur(gray1, (5,5),0)
    gray2=cv2.cvtColor(roi2, cv2.COLOR_BGR2GRAY)
    blur2=cv2.GaussianBlur(gray2, (5,5),0)
    (H,W)=frame.shape[:2]
    #Process 1st 30 frames to generate background
    if noFrame<=30:
        if noFrame==1:
            print('[INFO] Calibrating...')
        avgBgr(blur1, 1, Weight=0.5)
        avgBgr(blur2, 2, Weight=0.5)
        if noFrame==30:
            print('[INFO] Calibrated Successfully...')
    else:
        try:
            hand1=segment(blur1, 1)
            hand2=segment(blur2, 2)
            if hand1 is None:
                output1=0
            else:
                (thresholded1, segmented1)=hand1
                #Draw contour around hand1
                cv2.drawContours(roi1, [segmented1], -1, (120,255,0), 2)
                output1=getOutput(hand1, 1)
            if hand2 is None:
                output2=0
            else:
                (thresholded2, segmented2)=hand2
                #Draw contour around hand2
                cv2.drawContours(roi2, [segmented2], -1, (120,255,0), 2)
                output2=getOutput(hand2,2)
            # Displaying the 2 numbers extracted and there sum
            ans='{} + {} = {}'.format(output1, output2, output1+output2)
            cv2.putText(frame, ans, (150,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,255), 3, cv2.LINE_AA)
        except:
            pass
    #Draw rectangle to denote 2 ROIs
    cv2.rectangle(frame, (roiX1, roiY1),(roiX2, roiY2), (255,255,0), 2)
    cv2.rectangle(frame, (roiP1, roiQ1),(roiP2, roiQ2), (255,255,0), 2)
    #Saving the recording
    if writer==None and output!='':
        fourcc=cv2.VideoWriter_fourcc(*'MJPG')
        writer=cv2.VideoWriter(output, fourcc, 20, (frame.shape[1],frame.shape[0]),True)
    if writer is not None:
        writer.write(frame)
    noFrame+=1
    cv2.imshow('Frame', frame)
    key=cv2.waitKey(1)
    if key==27:
        break
writer.release()
cap.release()
cv2.destroyAllWindows()

[INFO] Calibrating...
[INFO] Calibrated Successfully...
